In [1]:
from symmer.projection import (
    S3_projection, QubitTapering, CS_VQE_LW, CS_VQE,
    StabilizerIdentification, ObservableBiasing, stabilizer_walk
)
from symmer.symplectic import *
from symmer.utils import gf2_gaus_elim, gf2_basis_for_gf2_rref, exact_gs_energy, QubitOperator_to_dict
from symmer.chem import geometry_from_pubchem, MoleculeBuilder, get_T2_mp2
from matplotlib import pyplot as plt
from functools import reduce
from openfermion import jordan_wigner
import json

with open('../data/molecule_data.json', 'r') as infile:
    molecule_geometries = json.load(infile)

In [20]:
speciesname = 'HF_3-21G_SINGLET'
mol_data = molecule_geometries[speciesname]
atoms  = mol_data['atoms']
coords = mol_data['coords']
basis = mol_data['basis']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)

# taper the Hamiltonian
taper_hamiltonian = QubitTapering(molecule.H_q)
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
hf_tapered = taper_hamiltonian.tapered_ref_state

# initiate stabilizer identification classes
CC_stabilizers = StabilizerIdentification(ucc_tap)

Molecule geometry:
F	0.0	0.0	0.093761
H	0.0	0.0	-0.84385

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -99.46021892473368
MP2 energy:  -99.58344905881171
CCSD energy: -99.58706294718117
FCI energy:  -99.58791845765421


Number of qubits: 22


In [21]:
ham_tap.n_qubits

19

In [3]:
# build CS-VQE model
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')
nc_before = cs_vqe.noncontextual_energy

In [4]:
S = CC_stabilizers.symmetry_basis_by_subspace_dimension(17)
print(S)

 1 ZIIIIIIIIIIIIIIIIIIIIIIIII 
 1 IZIIIIIIIIIIIIIIIIIIIIIIII 
 1 IIZIIIIIIIIIIIIIIIIIIIIIII 
 1 IIIZIIIIIIIIIIIIIIIIIIIIII 
 1 IIIIZIIIIIIIIIIIIIIIIIIIII 
 1 IIIIIZIIZZIZZIZIZIIZIZIZZI 
 1 IIIIIIZIZZIZIZIZIZIZIZIZIZ 
 1 IIIIIIIZZIIIIIZZZZIIIIZZII 
 1 IIIIIIIIIIZZIIIIZZIIZZIIII


In [ ]:
ham_cs  = cs_vqe.project_onto_subspace(S)
matrix  = ham_cs.to_sparse_matrix

In [ ]:
error_1 = exact_gs_energy(matrix)[0] - molecule.fci_energy
error_1

In [125]:
unique_comm, inverse = np.unique(cs_vqe.contextual_operator.commutes_termwise(S), axis=0, return_inverse=True)
single_anti = np.where(np.count_nonzero(~unique_comm, axis=1)==1)[0]
single_anti
#inverse==np.where(np.all(unique_comm, axis=1))[0]

array([1, 2])

In [126]:
which = 1
clique_0_mask = inverse==single_anti[which]
OG = ObservableGraph(
    cs_vqe.contextual_operator.symp_matrix[clique_0_mask], 
    cs_vqe.contextual_operator.coeff_vec[clique_0_mask]
)
commuting_cliques = OG.clique_cover(clique_relation='C', colouring_strategy='largest_first')
clique_0 = sorted(commuting_cliques.values(), key=lambda x:-x.n_terms)[0]
clique_0

0.060+0.000j IXIII +
-0.060+0.000j IXZZZ +
-0.038+0.000j ZXIII +
-0.010+0.000j ZXIIZ +
-0.010+0.000j ZXIZI +
0.010+0.000j ZXIZZ +
-0.010+0.000j ZXZII +
0.010+0.000j ZXZIZ +
0.010+0.000j ZXZZI +
0.038+0.000j ZXZZZ

In [127]:
C1 = S[int(np.where(~unique_comm[single_anti[which]])[0][0])]
clique_1_mask = np.where(
    C1.commutes_termwise(ham_tap) & np.all(~ham_tap.commutes_termwise(clique_0), axis=1)
)[1]
clique_1 = PauliwordOp(
    ham_tap.symp_matrix[clique_1_mask],
    ham_tap.coeff_vec[clique_1_mask]
)

In [128]:
#C1 = S[int(np.where(~unique_comm[single_anti[0]])[0][0])]

In [129]:
#(cs_vqe.noncontextual_operator + clique_0).is_noncontextual

In [130]:
noncon_op = clique_0 + clique_1# + S
universal_mask = np.where(np.all(ham_tap.commutes_termwise(noncon_op), axis=1))
noncon_op += PauliwordOp(
    ham_tap.symp_matrix[universal_mask], 
    ham_tap.coeff_vec[universal_mask]
)
noncon_op.is_noncontextual

True

In [131]:
cs_vqe.noncontextual_operator = noncon_op
ref_state = hf_tapered
self = cs_vqe
self.symmetry_generators, self.clique_operator = self.noncontextual_basis()
# Reconstruct the noncontextual Hamiltonian into its G and C(r) components
self.G_indices, self.r_indices, self.pauli_mult_signs = self.noncontextual_reconstruction()
# determine the noncontextual ground state - this updates the coefficients of the clique 
# representative operator C(r) and symmetry generators G with the optimal configuration
self.solve_noncontextual(ref_state)
# Determine the unitary partitioning rotations and the single Pauli operator that is rotated onto
if self.n_cliques > 0:
    self.unitary_partitioning_rotations, self.C0 = self.clique_operator.gen_seq_rotations()
    self.C0.coeff_vec[0] = round(self.C0.coeff_vec[0].real)
    
nc_after = cs_vqe.noncontextual_energy

In [132]:
cs_vqe.clique_operator

0.000 IXIII +
-1.000 IZZZZ

In [133]:
S_new = S - C1
error_2 = exact_gs_energy(cs_vqe.project_onto_subspace(S_new, enforce_clique_operator=True).to_sparse_matrix)[0] - molecule.fci_energy
error_2

0.000326505291548429

In [134]:
error_1 - error_2

6.423075404882184e-10

In [135]:
nc_before

-14.351880476202032

In [136]:
nc_after

-14.35188047620202

In [137]:
molecule.hf_energy

-14.351880476202023

In [138]:
S

 1 ZIZZZ 
 1 IZZZZ